In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,precision_score,confusion_matrix,accuracy_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

> preparation des données


In [3]:
df = pd.read_csv('FinalData.csv', index_col=0)
df.drop_duplicates(inplace=True)

df_Categorielle = df.drop(['Mt', 'Ewltp (g/km)', 'W (mm)', 'At1 (mm)', 'ec (cm3)', 'ep (KW)',
                           'Erwltp (g/km)', 'Fuel consumption '], axis=1)
df_quantitative = df[['Mt', 'Ewltp (g/km)', 'W (mm)', 'At1 (mm)', 'ec (cm3)', 'ep (KW)',
                       'Erwltp (g/km)', 'Fuel consumption ']]
df_quantitative = df_quantitative.fillna(df_quantitative.std()) 

df_Categorielle = df_Categorielle.join(pd.get_dummies(df_Categorielle[['Country','Ct','Ft','Cr']]))
df_Categorielle = df_Categorielle.drop(['Country','Ct','Ft','Cr'],axis=1)

le = LabelEncoder()
df_Categorielle['Mk'] = le.fit_transform(df_Categorielle['Mk'])
df_Categorielle['Mh'] = le.fit_transform(df_Categorielle['Mh'])
df_Categorielle['IT'] = le.fit_transform(df_Categorielle['IT'])

df_Final = df_Categorielle.join(df_quantitative)

data = df_Final.drop('Ewltp (g/km)',axis=1)
target = df_Final['Ewltp (g/km)']

def new_target(x):
    res = 0
    if(x <= 100) : res = 1
    elif((x > 100 ) & (x <= 120)):res = 2
    elif((x > 120) & (x <= 140)): res = 3
    elif((x > 140) & (x <= 160)): res = 4
    elif((x > 160) & (x <= 200)): res = 5
    elif((x > 200) & (x <= 250)): res = 6
    elif((x > 250)): res = 7
    return res

target = target.apply(new_target)
target

FileNotFoundError: [Errno 2] No such file or directory: 'FinalData.csv'

In [3]:
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.25, random_state=123)

In [4]:
scaler = StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

Decision tree

In [23]:
dt_clf = DecisionTreeClassifier(criterion='entropy', max_depth=4,random_state=123)
dt_clf.fit(X_train_scaled,y_train)

DecisionTreeClassifier(criterion='entropy', max_depth=4, random_state=123)

In [24]:
y_pred = dt_clf.predict(X_test_scaled)
pd.crosstab(y_test, y_pred, rownames=['reel'], colnames=['Predicted'])

Predicted,1,2,3,4,5,6,7
reel,,,,,,,
1,12069,7,124,1,3,1,0
2,0,104,5064,0,62,0,0
3,0,13,28492,1542,71,0,0
4,0,0,210,29060,301,1,0
5,0,1,0,305,20004,3317,0
6,0,0,0,0,172,10142,9
7,0,0,0,0,0,119,2530


In [25]:
cols = data.columns
result = {}

for i,j in zip(cols,dt_clf.feature_importances_):
    if j > 0 : 
        #print(i,j)
        result[i] = j

importante_feature = pd.DataFrame.from_dict(result,orient='index', columns=['Feature'])
print(importante_feature.sort_values(by="Feature",ascending=False))


                    Feature
Fuel consumption   0.812482
Mt                 0.104481
Ft_petrol          0.078758
Ft_diesel          0.004279


Gradient boosting

In [26]:
gb = GradientBoostingClassifier()
gb.fit(X_train_scaled,y_train)

GradientBoostingClassifier()

In [27]:
y_pred_gb = gb.predict(X_test_scaled)
pd.crosstab(y_test, y_pred, rownames=['reel'], colnames=['Predicted'])

Predicted,1,2,3,4,5,6,7
reel,,,,,,,
1,12069,7,124,1,3,1,0
2,0,104,5064,0,62,0,0
3,0,13,28492,1542,71,0,0
4,0,0,210,29060,301,1,0
5,0,1,0,305,20004,3317,0
6,0,0,0,0,172,10142,9
7,0,0,0,0,0,119,2530


KNN

In [28]:
knn = KNeighborsClassifier(n_neighbors=7,p=2, metric="minkowski")
knn.fit(X_train_scaled,y_train)

KNeighborsClassifier(n_neighbors=7)

In [29]:
y_pred_KNN = knn.predict(X_test_scaled)
pd.crosstab(y_test, y_pred_KNN, rownames=['reel'], colnames=['Predicted'])

Predicted,1,2,3,4,5,6,7
reel,,,,,,,
1,12189,16,0,0,0,0,0
2,15,4853,361,1,0,0,0
3,0,337,29274,507,0,0,0
4,0,3,421,28905,243,0,0
5,1,0,2,320,23191,113,0
6,0,0,0,0,177,10103,43
7,0,0,0,0,0,47,2602


Grid search : DecisionTreeClassifier

'criterion' : [ 'gini', 'entropy', 'log_loss'],
'max_depth':  [None, 2, 4, 6, 8, 10],
option : 
'max_features': [None, 'sqrt', 'log2', 0.2, 0.4, 0.6, 0.8],
'splitter': ['best', 'random']

In [7]:
param_gridDTC = {
    'criterion' : ['gini', 'entropy', 'log_loss'],
    'max_depth':  [None, 2, 4, 6, 8, 10],
}
grid_DTC = GridSearchCV(estimator=DecisionTreeClassifier(), param_grid=param_gridDTC, n_jobs=-1,
                        cv=2, verbose=1,refit= True)
grille = grid_DTC.fit(X_train_scaled,y_train)
print(pd.DataFrame.from_dict(grille.cv_results_).loc[:,['params', 'mean_test_score']])

print('le meilleur paramètre de DecisionTreeClassifier est :',grid_DTC.best_params_)

Fitting 2 folds for each of 18 candidates, totalling 36 fits
                                          params  mean_test_score
0       {'criterion': 'gini', 'max_depth': None}         0.979937
1          {'criterion': 'gini', 'max_depth': 2}         0.641460
2          {'criterion': 'gini', 'max_depth': 4}         0.912404
3          {'criterion': 'gini', 'max_depth': 6}         0.951649
4          {'criterion': 'gini', 'max_depth': 8}         0.959240
5         {'criterion': 'gini', 'max_depth': 10}         0.971777
6    {'criterion': 'entropy', 'max_depth': None}         0.980016
7       {'criterion': 'entropy', 'max_depth': 2}         0.628238
8       {'criterion': 'entropy', 'max_depth': 4}         0.910323
9       {'criterion': 'entropy', 'max_depth': 6}         0.974995
10      {'criterion': 'entropy', 'max_depth': 8}         0.978037
11     {'criterion': 'entropy', 'max_depth': 10}         0.979585
12  {'criterion': 'log_loss', 'max_depth': None}         0.980080
13     {'criter

Grid search : GradientBoostingClassifier
90 min 

In [9]:
param_gridGBC = {
    'learning_rate' : [0.1,0.2,0.3,0.4,0.5],
    'n_estimators':  [100, 200, 500],
}
grid_GBC = GridSearchCV(estimator=GradientBoostingClassifier(), param_grid=param_gridGBC, n_jobs=-1,
                        cv=2, verbose=1,refit= True)
grille = grid_GBC.fit(X_train_scaled,y_train)

print('le meilleur paramètre de GradientBoostingClassifier est :',grid_GBC.best_params_)
print(pd.DataFrame.from_dict(grille.cv_results_).loc[:,['params', 'mean_test_score']])


Fitting 2 folds for each of 15 candidates, totalling 30 fits
                                         params  mean_test_score
0   {'learning_rate': 0.1, 'n_estimators': 100}         0.978802
1   {'learning_rate': 0.1, 'n_estimators': 200}         0.980687
2   {'learning_rate': 0.1, 'n_estimators': 500}         0.982399
3   {'learning_rate': 0.2, 'n_estimators': 100}         0.980508
4   {'learning_rate': 0.2, 'n_estimators': 200}         0.981329
5   {'learning_rate': 0.2, 'n_estimators': 500}         0.982493
6   {'learning_rate': 0.3, 'n_estimators': 100}         0.981159
7   {'learning_rate': 0.3, 'n_estimators': 200}         0.981338
8   {'learning_rate': 0.3, 'n_estimators': 500}         0.554181
9   {'learning_rate': 0.4, 'n_estimators': 100}         0.977302
10  {'learning_rate': 0.4, 'n_estimators': 200}         0.977472
11  {'learning_rate': 0.4, 'n_estimators': 500}         0.977689
12  {'learning_rate': 0.5, 'n_estimators': 100}         0.949377
13  {'learning_rate': 0.5, 'n

Grid search : KNeighborsClassifier

parametres_kmean = {'n_neighbors':np.arange(2,41)}

In [8]:
param_gridKNN= {
    'metric' : ['minkowski','cityblock','cosine','euclidean','haversine','l1','l2','manhattan','nan_euclidean'],
    'n_neighbors':np.arange(2,10)
}


grid_KNN = GridSearchCV(estimator=KNeighborsClassifier(), param_grid=param_gridKNN, n_jobs=-1,
                        cv=2, verbose=1,refit= True)
grille = grid_KNN.fit(X_train_scaled,y_train)

print('le meilleur paramètre de KNeighborsClassifier est :',grid_KNN.best_params_)
print(pd.DataFrame.from_dict(grille.cv_results_).loc[:,['params', 'mean_test_score']])

Fitting 2 folds for each of 162 candidates, totalling 324 fits


c:\DEV\Anaconda\envs\DataScienceTest_101\Lib\site-packages\sklearn\model_selection\_validation.py:540: FitFailedWarning: 
180 fits failed out of a total of 324.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
27 fits failed with the following error:
Traceback (most recent call last):
  File "c:\DEV\Anaconda\envs\DataScienceTest_101\Lib\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\DEV\Anaconda\envs\DataScienceTest_101\Lib\site-packages\sklearn\base.py", line 1466, in wrapper
    estimator._validate_params()
  File "c:\DEV\Anaconda\envs\DataScienceTest_101\Lib\site-packages\sklearn\base.py", line 666, in _validate_params
    validate_paramete

le meilleur paramètre de KNeighborsClassifier est : {'metric': 'cityblock', 'n_neighbors': 3}
                                              params  mean_test_score
0          {'metric': 'minkowski', 'n_neighbors': 2}         0.973330
1          {'metric': 'minkowski', 'n_neighbors': 3}         0.975127
2          {'metric': 'minkowski', 'n_neighbors': 4}         0.972996
3          {'metric': 'minkowski', 'n_neighbors': 5}         0.973274
4          {'metric': 'minkowski', 'n_neighbors': 6}         0.971267
..                                               ...              ...
157  {'metric': '‘nan_euclidean', 'n_neighbors': 15}              NaN
158  {'metric': '‘nan_euclidean', 'n_neighbors': 16}              NaN
159  {'metric': '‘nan_euclidean', 'n_neighbors': 17}              NaN
160  {'metric': '‘nan_euclidean', 'n_neighbors': 18}              NaN
161  {'metric': '‘nan_euclidean', 'n_neighbors': 19}              NaN

[162 rows x 2 columns]
